In [1]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import os
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [14]:
df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',
                 sep=';',
                 parse_dates=['MESS_DATUM','MESS_DATUM_WOZ'],
                 date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H:%M"))
display(df.head, df.dtypes)

/var/folders/c5/491yhz3s1y93gvqdkz7t62p00000gn/T/ipykernel_37260/2147556409.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',


<bound method NDFrame.head of       mRID          Startzeit            Endzeit Resolution Position  Preis
0        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54
...    ...                ...                ...        ...      ...    ...
50039  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       90   9.61
50040  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       91   4.50
50041  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       92   0.10
50042  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       93   1.14
50043  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       94   0.00

[50044 rows x 6 columns]>

mRID           object
Startzeit      object
Endzeit        object
Resolution     object
Position       object
Preis         float64
dtype: object

In [ ]:
# Abfrage der Daten auf entso-e
# Ausfühle Informationen unter https://documenter.getpostman.com/view/7009892/2s93JtP3F6#3b383df0-ada2-49fe-9a50-98b1bb201c6b

energy_prices_url = 'https://web-api.tp.entsoe.eu/api?'

response = requests.get(url=energy_prices_url,
                        params=
                        {'securityToken': os.getenv('TOKEN'),
                         'DocumentType': 'A44',
                         'ProcessType':'A01',
                         'periodStart': '202501010000',
                         'periodEnd': '202512310000',
                         'outBiddingZone_Domain': '10Y1001A1001A82H',
                         'out_Domain': '10Y1001A1001A82H',
                         'in_Domain': '10Y1001A1001A82H',
                         'contract_MarketAgreement.type': 'A01'
                        })

# Antwort verarbeiten
if response.status_code == 200:
    # Direkte Nutzung der XML-Daten (ohne Dekomprimierung)
    xml_data = response.content.decode('utf-8')  # Bytes zu String dekodieren

    # XML-Daten parsen
    root = ET.fromstring(xml_data)

    # Namespace für die XML-Daten (falls vorhanden)
    namespaces = {
        'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'
    }

    # Liste für die Daten
    data = []

    # Durch alle TimeSeries iterieren
    for timeseries in root.findall('.//ns:TimeSeries', namespaces):
        # Sequenz prüfen
        sequence = timeseries.find('.//ns:classificationSequence_AttributeInstanceComponent.position', namespaces)
        if sequence is not None and sequence.text == '2':
            mrid = timeseries.find('ns:mRID', namespaces).text
            for period in timeseries.findall('.//ns:Period', namespaces):
                end = period.find('.//ns:timeInterval/ns:end', namespaces).text
                resolution = period.find('ns:resolution', namespaces).text
                for point in period.findall('.//ns:Point', namespaces):
                    position = point.find('ns:position', namespaces).text
                    price = point.find('ns:price.amount', namespaces).text

                    # Endzeit als Pandas-Datetime-Objekt speichern
                    end_datetime = pd.to_datetime(end).normalize()  # Uhrzeit auf 00:00:00 

                    data.append({
                        'mRID': mrid,
                        'Endzeit': end_datetime,  # Als Pandas-Datetime
                        'Resolution': resolution,
                        'Position': position,
                        'Preis': float(price)
                    })

    # DataFrame erstellen
    df = pd.DataFrame(data)

    # Ausgabe des DataFrames
    print(df)

    # Optional: DataFrame in CSV speichern
    #df.to_csv('day_ahead_preise_sequence2.csv', index=False)
else:
    print(f"Fehler bei der API-Anfrage: {response.status_code}")
    print(response.content)

      mRID                   Endzeit Resolution Position  Preis
0        1 2025-01-01 00:00:00+00:00      PT15M        1  22.40
1        1 2025-01-01 00:00:00+00:00      PT15M        2  -7.40
2        1 2025-01-01 00:00:00+00:00      PT15M        3 -10.16
3        1 2025-01-01 00:00:00+00:00      PT15M        4  -5.30
4        1 2025-01-01 00:00:00+00:00      PT15M        5   5.54
...    ...                       ...        ...      ...    ...
34605  729 2025-12-31 00:00:00+00:00      PT15M       92  68.40
34606  729 2025-12-31 00:00:00+00:00      PT15M       93  79.33
34607  729 2025-12-31 00:00:00+00:00      PT15M       94  73.85
34608  729 2025-12-31 00:00:00+00:00      PT15M       95  66.00
34609  729 2025-12-31 00:00:00+00:00      PT15M       96  57.85

[34610 rows x 5 columns]


In [115]:
print(df.loc[:,'Position'].unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100']


In [73]:
# Aufteilen des Dataframes in zwei Stück gefiltert nach Resolution PT15M und PT60M

# Erstellen des Filters für PT15M und PT60M
filter_resolution_PT15M = df['Resolution'] == 'PT15M'
filter_resolution_PT60M = df['Resolution'] == 'PT60M'
 
 # Erstellen zweier neuer Dataframes
df_PT15M = df[filter_resolution_PT15M].copy()
df_PT60M = df[filter_resolution_PT60M].copy()

# Anzeigen der ersten 5 Zeilen
print(df_PT15M.head())
print()
print(df_PT60M.head())
 

  mRID          Startzeit            Endzeit Resolution Position  Preis
0    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54

    mRID          Startzeit            Endzeit Resolution Position  Preis
96     2  2024-12-31T23:00Z  2025-01-01T23:00Z      PT60M        1   2.16
97     2  2024-12-31T23:00Z  2025-01-01T23:00Z      PT60M        2   1.60
98     2  2024-12-31T23:00Z  2025-01-01T23:00Z      PT60M        3   0.00
99     2  2024-12-31T23:00Z  2025-01-01T23:00Z      PT60M        4  -0.01
100    2  2024-12-31T23:00Z  2025-01-01T23:00Z      PT60M        6  -0.06


In [74]:
print(df_PT15M[-1:])

      mRID          Startzeit            Endzeit Resolution Position  Preis
50043  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       94    0.0


In [76]:
print(df_PT60M.dtypes)
print(df_dwd.dtypes)

mRID           object
Startzeit      object
Endzeit        object
Resolution     object
Position       object
Preis         float64
dtype: object
STATIONS_ID                int64
MESS_DATUM        datetime64[ns]
QN_592                     int64
ATMO_LBERG               float64
FD_LBERG                 float64
FG_LBERG                 float64
SD_LBERG                   int64
ZENIT                    float64
MESS_DATUM_WOZ    datetime64[ns]
eor                       object
dtype: object


In [75]:
df_PT15M['Position'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100'],
      dtype=object)

In [ ]:
PT15M_filter = df_PT15M['Startzeit'].str.startswith("2025-10-25")
df_PT15M.to_excel("tabelle_check.xlsx")


In [84]:
from entsoe import EntsoePandasClient

client = EntsoePandasClient(api_key=os.getenv('TOKEN'))

start = pd.Timestamp('20250101', tz='Europe/Brussels')
end = pd.Timestamp('20260101', tz='Europe/Brussels')
country_code = 'DE_LU'  # Belgium
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A01'

# methods that return Pandas Series
df_new = client.query_day_ahead_prices(country_code, start=start, end=end)

In [85]:
df_new

2025-01-01 00:00:00+01:00     2.16
2025-01-01 01:00:00+01:00     1.60
2025-01-01 02:00:00+01:00     0.00
2025-01-01 03:00:00+01:00    -0.01
2025-01-01 04:00:00+01:00    -0.01
                             ...  
2025-12-31 23:00:00+01:00    86.30
2025-12-31 23:15:00+01:00    76.48
2025-12-31 23:30:00+01:00    75.75
2025-12-31 23:45:00+01:00    67.26
2026-01-01 00:00:00+01:00    63.64
Length: 15387, dtype: float64